In [1]:
import pandas as pd
import numpy as np

In [2]:
feattype = 'raw'
coltype = 'first'
train = pd.read_pickle('../data/processed_'+feattype+'/train_'+feattype+'_'+coltype+'.pickle')
test = pd.read_pickle('../data/processed_'+feattype+'/test_'+feattype+'_'+coltype+'.pickle')

In [2]:
ext = pd.read_pickle('../data/external_features//train_external.pickle')
text = pd.read_pickle('../data/external_features//test_external.pickle')
allext = ext.append(text)
#traindf1 = pd.read_pickle('../data/processed_'+feattype+'/train_'+feattype+'_row.pickle')
#querydf = pd.read_csv('../data/raw/data_set_phase1/train_queries.csv')
#plandf = pd.read_csv('../data/raw/data_set_phase1/train_plans.csv')


In [3]:
train.shape

(400000, 177)

In [6]:
train[train["pid"] == -1].iloc[4]

sid                             415664
click_time         2018-10-05 10:04:49
click_mode                           2
pid                                 -1
req_time           2018-10-05 10:04:46
o_long                          116.31
o_lat                            40.03
d_long                          116.37
d_lat                            39.86
distance_query                0.180278
weekday                              4
hour                                10
PID_MainComp_0                 5.15078
PID_MainComp_1              -0.0309079
PID_MainComp_2               -0.132038
PID_MainComp_3              -0.0347558
PID_MainComp_4               0.0112912
PID_MainComp_5                0.325687
PID_MainComp_6                -0.60347
PID_MainComp_7               -0.571777
PID_MainComp_8               -0.678642
PID_MainComp_9               -0.670281
PID_MainComp_10              -0.186577
PID_MainComp_11              -0.162506
PID_MainComp_12              -0.200601
PID_MainComp_13          

In [13]:
train.shape, test.shape, ext.shape, text.shape, trainr.shape

((400000, 195), (100000, 195), (400000, 21), (100000, 21), (400000, 177))

In [4]:
dict(train.iloc[10])

{'sid': 2629085,
 'click_time': '2018-10-12 16:28:13',
 'click_mode': 3,
 'distance_plan': 9546,
 'eta': 4553,
 'price': 400.0,
 'transport_mode': 1,
 'plan_time_x': '2018-10-12 16:27:55',
 'pid': 203797.0,
 'req_time': Timestamp('2018-10-12 16:27:55'),
 'o_long': 116.35,
 'o_lat': 40.08,
 'd_long': 116.33,
 'd_lat': 40.03,
 'distance_query': 0.05385164807134093,
 'weekday': 4,
 'hour': 16,
 'plan_time_y': '2018-10-12 16:27:55',
 'PID_MainComp_0': 1.704739783115594,
 'PID_MainComp_1': -0.4177136334797418,
 'PID_MainComp_2': -1.1324949263837258,
 'PID_MainComp_3': -0.6521039627208052,
 'PID_MainComp_4': -0.4392447761906962,
 'PID_MainComp_5': -0.9398386527111019,
 'PID_MainComp_6': -0.08353287941948112,
 'PID_MainComp_7': 1.042147253660637,
 'PID_MainComp_8': -0.5090381406784733,
 'PID_MainComp_9': -0.45492200427524376,
 'PID_MainComp_10': 0.13182880673274722,
 'PID_MainComp_11': -0.025284097312881275,
 'PID_MainComp_12': -0.48044921816239083,
 'PID_MainComp_13': -0.3669746493846503,
 '

In [7]:
nandf = newdf[newdf.isna().any(axis=1)]

In [8]:
print(nandf.isna().any(axis=0))

sid                 False
click_time           True
click_mode          False
pid_x               False
req_time_x          False
o_long              False
o_lat               False
d_long              False
d_lat               False
distance_query      False
weekday             False
hour                False
PID_MainComp_0      False
PID_MainComp_1      False
PID_MainComp_2      False
PID_MainComp_3      False
PID_MainComp_4      False
PID_MainComp_5      False
PID_MainComp_6      False
PID_MainComp_7      False
PID_MainComp_8      False
PID_MainComp_9      False
PID_MainComp_10     False
PID_MainComp_11     False
PID_MainComp_12     False
PID_MainComp_13     False
PID_MainComp_14     False
PID_MainComp_15     False
PID_MainComp_16     False
PID_MainComp_17     False
                    ...  
dist_9              False
price_9             False
eta_9               False
dist_10             False
price_10            False
eta_10              False
dist_11             False
price_11    

In [20]:
train.to_pickle('../data/processed_all/train_all_last.pickle')
test.to_pickle('../data/processed_all/test_all_last.pickle')